In [1]:
import pandas as pd
import numpy as np
import itertools
from entsoe import EntsoePandasClient
from entsoe.geo import utils
import os
import entsoe.geo.geojson

import os
import sys
giditoolbox_path = '../day_ahead_arbitrage_map/giditoolbox'

sys.path.insert(1, giditoolbox_path)

import battery


### Query day ahead prices based on bidding zone, and save as csv files.

In [2]:
def create_df_daily_spread_avg_country(df_zone, zone):
    df = df_zone.copy()
    df['date'] = df.index.date
    df_daily_max = df.groupby('date').max()
    df_daily_min = df.groupby('date').min()
    df_daily_spread = pd.DataFrame(df_daily_max - df_daily_min)
    df_daily_spread.index = pd.to_datetime(df_daily_spread.index)
    df_daily_spread['Year'] = df_daily_spread.index.year
    df_daily_spread_avg_country = df_daily_spread.groupby('Year').mean()
    df_daily_spread_avg_country['zoneName'] = zone
    df_daily_spread_avg_country = df_daily_spread_avg_country.rename(columns={0: "Average daily spread [€]"})
    return df_daily_spread_avg_country  

In [3]:
def run_EA_optimization(df_zone, battery, daily_cycle_limit):
    years = df_zone.index.year.unique().astype(str)
    revenues = np.zeros(len(years))
    for year_idx, year in enumerate(years):
        print(year)
        df_zone_year = df_zone.loc[year]
        zone_year_nr_days = len(np.unique(df_zone_year.index.date))
        prices = df_zone_year[0]
        df_optimization_results = battery.applications.arbitrage(prices, 
                                                    max_efc = zone_year_nr_days*daily_cycle_limit,
                                                    perfect_foresight=True
        )
        df_zone_year_revenue = df_optimization_results['transactions_eur'].sum()
        revenues[year_idx] = df_zone_year_revenue
    return revenues
    

In [4]:
def remove_incomplete_years(df_zone):
    years = df_zone.index.year.unique().astype(str)
    for year in years:
        df_zone_year = df_zone.loc[year]
        if len(df_zone_year) < 4500:
            df_zone = df_zone.loc[~df_zone.index.isin(df_zone_year.index)]
    return df_zone

In [9]:
api_key = "cc642494-539b-463a-b8e5-c29f406e5521"
client = EntsoePandasClient(api_key=api_key)


start = pd.Timestamp('20180101', tz="UTC")
end = pd.Timestamp('20190101', tz="UTC")

geojson_files = os.listdir("C:\\Users\\ricky\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\entsoe\\geo\\geojson")
zones = [zone_file.split(".")[0] for zone_file in geojson_files]
zones = ["DE_AT_LU"]

df_daily_spread_avg = pd.DataFrame()
df_arbitrage_revenues = pd.DataFrame()

battery_capacities = [1, 2] #h
daily_cycle_limits = [1, 2, 3]

for zone in zones:
        print(zone)
        df_zone = client.query_day_ahead_prices(zone, start=start, end=end)
        df_zone = pd.DataFrame(df_zone)
        df_zone = remove_incomplete_years(df_zone)
        df_daily_spread_avg_country = create_df_daily_spread_avg_country(df_zone, zone)
        df_daily_spread_avg = pd.concat([df_daily_spread_avg, df_daily_spread_avg_country])

        for battery_capacity, daily_cycle_limit in itertools.product(battery_capacities, daily_cycle_limits):
                battery_object = battery.Battery(max_power=1, net_capacity=battery_capacity)
                revenues = run_EA_optimization(df_zone, battery_object, daily_cycle_limit)
                df_arbitrage_revenues_cap_limit = pd.DataFrame(data={"Year": df_zone.index.year.unique() ,"Revenue [€/MWp/year]": revenues})
                df_arbitrage_revenues_cap_limit['zoneName'] = zone
                df_arbitrage_revenues_cap_limit['battery_capacity'] = battery_capacity
                df_arbitrage_revenues_cap_limit['daily_cycle_limit'] = daily_cycle_limit
                df_arbitrage_revenues = pd.concat([df_arbitrage_revenues, df_arbitrage_revenues_cap_limit])

df_revenues_spreads = pd.merge(df_arbitrage_revenues, df_daily_spread_avg, how="right", on=["Year", "zoneName"])


DE_AT_LU
2018
2018
2018
2018
2018
2018


In [6]:
df_revenues_spreads

,Year,Revenue [€/MWp/year],battery_capacity,daily_cycle_limit,Average daily spread [€],zoneName


In [7]:
start = pd.Timestamp('20171231', tz="UTC")
end = pd.Timestamp('20190101', tz="UTC")
zone = "AT"
df_zone = client.query_day_ahead_prices(zone, start=start, end=end)
df_zone

2018-10-01 00:00:00+02:00    59.53
2018-10-01 01:00:00+02:00    56.10
2018-10-01 02:00:00+02:00    51.41
2018-10-01 03:00:00+02:00    47.38
2018-10-01 04:00:00+02:00    47.59
                             ...  
2018-12-31 21:00:00+01:00    45.15
2018-12-31 22:00:00+01:00    49.15
2018-12-31 23:00:00+01:00    34.09
2019-01-01 00:00:00+01:00    33.48
2019-01-01 01:00:00+01:00    39.76
Length: 2211, dtype: float64

In [11]:
df_zone = client.query_day_ahead_prices("DE_AT_LU", start=start, end=end)

In [7]:
df_revenues_spreads.to_csv("de_lu_revenues_spreads_2018.csv", index=False)

In [15]:
df_zone.loc["2018"]

2018-01-01 00:00:00+01:00    -5.27
2018-01-01 01:00:00+01:00   -29.99
2018-01-01 02:00:00+01:00   -56.65
2018-01-01 03:00:00+01:00   -63.14
2018-01-01 04:00:00+01:00   -64.62
                             ...  
2018-09-30 19:00:00+02:00    74.21
2018-09-30 20:00:00+02:00    71.31
2018-09-30 21:00:00+02:00    65.31
2018-09-30 22:00:00+02:00    65.08
2018-09-30 23:00:00+02:00    60.87
Length: 6551, dtype: float64

In [11]:
df_revenues_spreads = pd.merge(df_arbitrage_revenues, df_daily_spread_avg, how="right", on=["Year", "zoneName"])

In [35]:
df_arbitrage_revenues['revenue_per_MWh'] = df_arbitrage_revenues


ValueError: Cannot set a DataFrame with multiple columns to the single column revenue_per_MWh

In [12]:
df_revenues_spreads

,Year,Revenue [€/MWp/year],zoneName,battery_capacity,daily_cycle_limit,Average daily spread [€]
0,2019,7661.937401,AT,1,1,26.716767
1,2019,8658.684859,AT,1,2,26.716767
2,2019,8658.684859,AT,1,3,26.716767
3,2019,14111.381598,AT,2,1,26.716767
4,2019,15508.667013,AT,2,2,26.716767
...,...,...,...,...,...,...
1537,2022,69415.493408,SK,1,2,199.861808
1538,2022,69415.493408,SK,1,3,199.861808
1539,2022,109604.005348,SK,2,1,199.861808
1540,2022,122769.219807,SK,2,2,199.861808


In [20]:
df_daily_spread_avg

,0,zone
year,,
2022,163.509644,AT


In [47]:
df_daily_spread_avg.reset_index()

,year,average_daily_spread,zone
0,2020,28.246311,AT
1,2021,72.315452,AT
2,2022,163.509644,AT
3,2020,30.292158,BE
4,2021,82.554575,BE
5,2022,197.343726,BE


In [102]:
df_revenues_spreads = pd.merge(df_arbitrage_revenues, df_daily_spread_avg, how="right", on=["year", "zone"])

In [103]:
df_revenues_spreads = df_revenues_spreads.rename(columns={"zone": "zoneName"})
df_revenues_spreads

,year,revenue,zoneName,battery_capacity,daily_cycle_limit,average_daily_spread
0,2020,8629.535642,AT,1,1,28.246311
1,2020,9807.649436,AT,1,2,28.246311
2,2020,9807.649436,AT,1,3,28.246311
3,2020,15576.152994,AT,2,1,28.246311
4,2020,17298.026029,AT,2,2,28.246311
...,...,...,...,...,...,...
67,2022,34170.682614,CH,1,2,126.422493
68,2022,34170.682614,CH,1,3,126.422493
69,2022,59203.336730,CH,2,1,126.422493
70,2022,62151.947166,CH,2,2,126.422493


In [24]:
df_daily_spread_avg.loc[2018]

,0,zone
year,,
2018,36.324674,AT
2018,42.989671,BE


In [59]:
geo_df = pd.merge(df_revenues_spreads, geo_df, how="right", on="zoneName")

In [60]:
geo_df

,year,revenue,zoneName,battery_capacity,daily_cycle_limit,average_daily_spread,geometry,value
0,2020,8629.535642,AT,1,1,28.246311,"POLYGON ((16.94504 48.60417, 16.95434 48.55740...",1
1,2021,20813.896596,AT,1,1,72.315452,"POLYGON ((16.94504 48.60417, 16.95434 48.55740...",1
2,2022,46594.839039,AT,1,1,163.509644,"POLYGON ((16.94504 48.60417, 16.95434 48.55740...",1
3,2020,9575.075581,BE,1,1,30.292158,"POLYGON ((2.52180 51.08754, 2.54200 51.09687, ...",2
4,2021,26817.500978,BE,1,1,82.554575,"POLYGON ((2.52180 51.08754, 2.54200 51.09687, ...",2
5,2022,61999.990091,BE,1,1,197.343726,"POLYGON ((2.52180 51.08754, 2.54200 51.09687, ...",2


In [70]:
geo_df

,year,revenue,zoneName,battery_capacity,daily_cycle_limit,average_daily_spread,geometry
0,2020,8629.535642,AT,1,1,28.246311,"POLYGON ((16.94504 48.60417, 16.95434 48.55740..."
1,2020,9575.075581,BE,1,1,30.292158,"POLYGON ((2.52180 51.08754, 2.54200 51.09687, ..."


In [90]:
# geo_df = utils.load_zones(zones, pd.Timestamp('20210101'))
geo_df

,zoneName,geometry,year,revenue,battery_capacity,daily_cycle_limit,average_daily_spread,value
0,AT,"POLYGON ((16.94504 48.60417, 16.95434 48.55740...",2020,8629.535642,1,1,28.246311,1
1,BE,"POLYGON ((2.52180 51.08754, 2.54200 51.09687, ...",2020,9575.075581,1,1,30.292158,2


In [106]:
df_revenues_spreads.loc[
    (df_revenues_spreads['year'] == 2020) &
    (df_revenues_spreads['battery_capacity'] == 1)
]

,year,revenue,zoneName,battery_capacity,daily_cycle_limit,average_daily_spread
0,2020,8629.535642,AT,1,1,28.246311
1,2020,9807.649436,AT,1,2,28.246311
2,2020,9807.649436,AT,1,3,28.246311
18,2020,9575.075581,BE,1,1,30.292158
19,2020,11476.708583,BE,1,2,30.292158
20,2020,11478.452161,BE,1,3,30.292158
36,2020,18901.771211,BG,1,1,62.432705
37,2020,21390.277840,BG,1,2,62.432705
38,2020,21390.277840,BG,1,3,62.432705
54,2020,6252.558385,CH,1,1,21.567678


In [13]:
df_revenues_spreads.to_csv("revenues_spreads.csv", index=False)

In [107]:
import plotly.express as px
import pandas as pd

# geo_df =

geo_df = utils.load_zones(zones, pd.Timestamp('20210101'))
df_revenues_spreads_year = df_revenues_spreads[
    (df_revenues_spreads['year'] == 2020) &
    (df_revenues_spreads['battery_capacity'] == 1) &
    (df_revenues_spreads['daily_cycle_limit'] == 2)
]
geo_df = pd.merge(geo_df, df_revenues_spreads_year, how="left", on="zoneName")
geo_df = geo_df.set_index('zoneName')
# geo_df['value'] = range(1,len(geo_df)+1)
fig = px.choropleth(geo_df,
                   geojson=geo_df.geometry,
                   locations=geo_df.index,
                   color="average_daily_spread",
                   projection="mercator",
                   color_continuous_scale='inferno')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [86]:
type(geo_df.geometry)

geopandas.geoseries.GeoSeries